This is an assignment for my thinkful data science course - "Data Cleaning 2"
Raj Prasad
May 2019


[html version](https://daddyprasad5.github.io/Data_Cleaning_2.html) - with all the code hidden away for a quick read

[jupyter notebook version](https://github.com/daddyprasad5/thinkful/blob/master/Data_Cleaning_2.ipynb) - with all the code exposed in an interactive notebook

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
import math
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline



In [ ]:
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

used_df = pd.read_sql_query('select * from useducation',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [ ]:
used_df.info()

In [ ]:
used_df.tail()

In [ ]:
used_df.nunique()


In [ ]:
#check for null-like entries in categorical variables
u_state = list(used_df.STATE.unique())
u_year = list(used_df.YEAR.unique())
#pd.DataFrame([u_state, u_year]).transpose()
print(u_state)
print(u_year)

In [ ]:
used_df.describe()

1a. Determine all the variable types 

These are nominal: 
* PRIMARY_KEY                     1487
* STATE                             80

This is ordinal:
* YEAR                              26

These are 


These are ratio: 
* ENROLL                          1224
* TOTAL_REVENUE                   1274
* FEDERAL_REVENUE                 1275
* STATE_REVENUE                   1251
* LOCAL_REVENUE                   1275
* TOTAL_EXPENDITURE               1275
* INSTRUCTION_EXPENDITURE         1275
* SUPPORT_SERVICES_EXPENDITURE    1275
* OTHER_EXPENDITURE               1222
* CAPITAL_OUTLAY_EXPENDITURE      1275
* GRADES_PK_G                     1261
* GRADES_KG_G                     1348
* GRADES_4_G                      1340
* GRADES_8_G                      1347
* GRADES_12_G                     1342
* GRADES_1_8_G                    1360
* GRADES_9_12_G                   1358
* GRADES_ALL_G                    1318
* AVG_MATH_4_SCORE                 535
* AVG_MATH_8_SCORE                 531
* AVG_READING_4_SCORE              532
* AVG_READING_8_SCORE              497

1b  find the fraction of the missing values for each variable.

In [ ]:
n = len(used_df)
cols = used_df.columns
num_null = [len(used_df[used_df[col].isnull()]) / n for col in used_df]
pd.DataFrame([cols, num_null]).transpose()

2. Notice that the data has a time dimension (year). For this assignment, forget about time and treat all the observations as if they're from the same year. Choose a strategy to deal with the missing values for each variables. For which variables would filling in the missing values with some value make sense? For which might tossing out the records entirely make sense?

If there's no order (as year would impose), then I would use the mean or median for all the continuouse variables that are null.  There are too many to throw out any record with  a null. I'll use mean. 

In [ ]:
used_df.columns

In [ ]:
col_list = ['ENROLL', 'TOTAL_REVENUE',
       'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE',
       'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
       'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE',
       'CAPITAL_OUTLAY_EXPENDITURE', 'GRADES_PK_G', 'GRADES_KG_G',
       'GRADES_4_G', 'GRADES_8_G', 'GRADES_12_G', 'GRADES_1_8_G',
       'GRADES_9_12_G', 'GRADES_ALL_G', 'AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE',
       'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE'] 

for col in col_list:
    ave = used_df[col].mean()
    used_df[(col+"v1")] = used_df[(col)].fillna(ave, inplace=False)

In [ ]:
n = len(used_df)
cols = used_df.columns
num_null = [len(used_df[used_df[col].isnull()]) / n for col in used_df]
pd.DataFrame([cols, num_null]).transpose()

In [ ]:
years = used_df["YEAR"].unique()


#loop through each column in the dataframe
for col in col_list:
    new_col = []
    year_ave = []
    for y in years: 
        year_ave.append(np.nanmean(used_df[used_df["YEAR"] == y][col]))
    year_ave_dict = dict(zip(years, year_ave))
    overall_ave = used_df[col].mean()
    for row in used_df.itertuples():
        curr_val = getattr(row, col)
        curr_year = getattr(row, "YEAR")
        curr_year_ave = year_ave_dict.get(curr_year)
        if np.isnan(curr_val): 
            if np.isnan(curr_year_ave): 
                new_col.append(overall_ave)
            else: 
                new_col.append(curr_year_ave)
        else: new_col.append(curr_val)
    used_df[col + "v2"] = new_col

In [ ]:
n = len(used_df)
cols = used_df.columns
num_null = [len(used_df[used_df[col].isnull()]) / n for col in used_df]
pd.DataFrame([cols, num_null]).transpose()

In [ ]:
used_df["new_pk"] = used_df.STATE + used_df.YEAR.map(str)
used_df.sort_values(by="new_pk").head()

In [ ]:
new_used_df = used_df.interpolate()
    

In [ ]:
n = len(new_used_df)
cols = new_used_df.columns
num_null = [len(new_used_df[new_used_df[col].isnull()]) / n for col in used_df]
pd.DataFrame([cols, num_null]).transpose()

In [ ]:
sns.lineplot(x="YEAR", y="TOTAL_REVENUE", data=new_used_df[used_df.STATE == "ALABAMA"], ci=None)
sns.lineplot(x="YEAR", y="TOTAL_REVENUEv1", data=used_df[used_df.STATE == "ALABAMA"], ci=None)
sns.lineplot(x="YEAR", y="TOTAL_REVENUEv2", data=used_df[used_df.STATE == "ALABAMA"], ci=None)
plt.legend(["interpolate", "total average", "average for year"])

In [ ]:
sns.lineplot(x="YEAR", y="AVG_READING_8_SCORE", data=new_used_df[used_df.STATE == "ALABAMA"], ci=None)
sns.lineplot(x="YEAR", y="AVG_READING_8_SCOREv1", data=used_df[used_df.STATE == "ALABAMA"], ci=None)
sns.lineplot(x="YEAR", y="AVG_READING_8_SCOREv2", data=used_df[used_df.STATE == "ALABAMA"], ci=None)
plt.legend(["interpolate", "total average", "average for year"])

In [ ]:
sns.lineplot(x="YEAR", y="ENROLL", data=new_used_df[used_df.STATE == "ALABAMA"], ci=None)
sns.lineplot(x="YEAR", y="ENROLLv1", data=used_df[used_df.STATE == "ALABAMA"], ci=None)
sns.lineplot(x="YEAR", y="ENROLLv2", data=used_df[used_df.STATE == "ALABAMA"], ci=None)
plt.legend(["interpolate", "total average", "average for year"])

In [ ]:
used_df.TOTAL_REVENUEv1.head()


In [ ]:
new_used_df.head()